# Import libraries

In [1]:
import os
import warnings

import numpy as np
import pandas as pd

import json

import re

from tqdm import tqdm

from itertools import combinations

In [2]:
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', None)

In [3]:
path_to_save = './../results/03a_features'

# Loading data

In [4]:
path_to_ftir_data_3800_2500 = './../data/processed_data/dataset_3800_2500_extracted.csv'
path_to_ftir_data_1900_800= './../data/processed_data/dataset_1900_800_extracted.csv'

## FTIR-features (3800-2500 cm$^{-1}$)

In [5]:
dataset_3800_2500 = pd.read_csv(path_to_ftir_data_3800_2500, sep=';', index_col=0)

print(dataset_3800_2500.shape)
dataset_3800_2500.head(3)

(74, 12)


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.491720,1.0,0.088051,0.526201,1.579770,0.494443,0.983311,0.075225,Train,$> 0$,$d > 5$,2
division_1_size_5,0.406937,1.0,0.028000,0.477909,1.434937,0.396989,0.980473,0.034739,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.618445,1.0,0.078392,0.535789,1.696837,0.581733,0.980981,0.066886,Train,$< 3$,$2 < d \leq 5$,1


## FTIR-features (1900-800 cm$^{-1}$)

In [6]:
dataset_1900_800 = pd.read_csv(path_to_ftir_data_1900_800, sep=';', index_col=0)

print(dataset_1900_800.shape)
dataset_1900_800.head(3)

(74, 26)


,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.047800,0.681294,0.388923,0.455066,0.760582,1.0,0.423273,0.424162,3.756938,0.426619,1.423273,0.571735,0.760582,0.363338,1.573083,0.047238,0.658256,0.399497,0.447733,0.738860,0.936425,0.417470,Train,$> 0$,$d > 5$,2
division_1_size_5,0.091438,0.606138,0.416593,0.427988,0.687458,1.0,0.384475,0.378730,3.614090,0.378004,1.384475,0.502368,0.687458,0.329999,1.542157,0.092933,0.571531,0.426105,0.424918,0.667947,0.926718,0.378463,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.108127,0.836176,0.514367,0.400950,0.696553,1.0,0.368753,0.436266,3.924926,0.429362,1.368753,0.523513,0.696553,0.406988,1.859620,0.111725,0.776148,0.526741,0.401435,0.678774,0.951396,0.369746,Train,$< 3$,$2 < d \leq 5$,1


# Feature engineering

## Data wrangling

In [7]:
features_3800_2500 = dataset_3800_2500.loc[:, ~dataset_3800_2500.columns.isin(['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class'])]

print(features_3800_2500.shape)
features_3800_2500.head(3)

(74, 8)


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$
division_1_size_bulk,0.491720,1.0,0.088051,0.526201,1.579770,0.494443,0.983311,0.075225
division_1_size_5,0.406937,1.0,0.028000,0.477909,1.434937,0.396989,0.980473,0.034739
division_1_size_3,0.618445,1.0,0.078392,0.535789,1.696837,0.581733,0.980981,0.066886


In [8]:
features_1900_800 = dataset_1900_800.loc[:, ~dataset_1900_800.columns.isin(['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class'])]

print(features_1900_800.shape)
features_1900_800.head(3)

(74, 22)


,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$
division_1_size_bulk,0.047800,0.681294,0.388923,0.455066,0.760582,1.0,0.423273,0.424162,3.756938,0.426619,1.423273,0.571735,0.760582,0.363338,1.573083,0.047238,0.658256,0.399497,0.447733,0.738860,0.936425,0.417470
division_1_size_5,0.091438,0.606138,0.416593,0.427988,0.687458,1.0,0.384475,0.378730,3.614090,0.378004,1.384475,0.502368,0.687458,0.329999,1.542157,0.092933,0.571531,0.426105,0.424918,0.667947,0.926718,0.378463
division_1_size_3,0.108127,0.836176,0.514367,0.400950,0.696553,1.0,0.368753,0.436266,3.924926,0.429362,1.368753,0.523513,0.696553,0.406988,1.859620,0.111725,0.776148,0.526741,0.401435,0.678774,0.951396,0.369746


In [9]:
initial_features_df = pd.concat([features_3800_2500, features_1900_800], axis=1)

print(initial_features_df.shape)
initial_features_df.head(3)

(74, 30)


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$
division_1_size_bulk,0.491720,1.0,0.088051,0.526201,1.579770,0.494443,0.983311,0.075225,0.047800,0.681294,0.388923,0.455066,0.760582,1.0,0.423273,0.424162,3.756938,0.426619,1.423273,0.571735,0.760582,0.363338,1.573083,0.047238,0.658256,0.399497,0.447733,0.738860,0.936425,0.417470
division_1_size_5,0.406937,1.0,0.028000,0.477909,1.434937,0.396989,0.980473,0.034739,0.091438,0.606138,0.416593,0.427988,0.687458,1.0,0.384475,0.378730,3.614090,0.378004,1.384475,0.502368,0.687458,0.329999,1.542157,0.092933,0.571531,0.426105,0.424918,0.667947,0.926718,0.378463
division_1_size_3,0.618445,1.0,0.078392,0.535789,1.696837,0.581733,0.980981,0.066886,0.108127,0.836176,0.514367,0.400950,0.696553,1.0,0.368753,0.436266,3.924926,0.429362,1.368753,0.523513,0.696553,0.406988,1.859620,0.111725,0.776148,0.526741,0.401435,0.678774,0.951396,0.369746


## Functions for processing

### Spectral region determination

In [10]:
def extract_center(name: str):
    nums = re.findall(r"\d+", name)
    return int(nums[0]) if nums else None

def get_interval_tag(center):
    if center is None:
        return None
    if 2500 <= center <= 3800:
        return "high"
    elif 800 <= center <= 1900:
        return "low"
    return None

### Features construction

In [11]:
def generate_transformed_features(data, feature_list, tolerance=50, zero_substitute=1e-8):
    data_transformed = pd.DataFrame(index=data.index)

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def wrap_log(col):
        return fr"$\log {clean(col)}$"

    def wrap_pow(col, power):
        return fr"$({clean(col)})^{{{power}}}$"

    def wrap_div(col1, col2):
        return fr"$\frac{{{clean(col1)}}}{{{clean(col2)}}}$"

    def extract_center(name):
        nums = re.findall(r'\d+', name)
        return int(nums[0]) if nums else None

    def get_interval_tag(center):
        if center is None:
            return None
        if 2500 <= center <= 3800:
            return "high"
        elif 800 <= center <= 1900:
            return "low"
        return None

    def extract_center_and_type(name):
        center = extract_center(name)
        is_avg = r'\overline{I}' in name or '±' in name
        return center, is_avg

    def is_near_duplicate(f1, f2):
        c1, avg1 = extract_center_and_type(f1)
        c2, avg2 = extract_center_and_type(f2)
        if c1 is None or c2 is None:
            return False
        return abs(c1 - c2) <= tolerance and (avg1 or avg2)

    def get_interval_kind(name: str):
        if r"\overline{I}" in name or "±" in name:
            return "average"
        if r"\Sigma I_{p," in name:
            return "sum"
        return None
    
        np.seterr(divide='ignore', invalid='ignore')

    for col in feature_list:
        base = data[col].copy()
        base_safe = base.replace(0, zero_substitute)

        data_transformed[wrap_log(col)] = np.log(base_safe)

        for power in [-1]:
            transformed = np.power(base_safe, power)
            transformed.replace([np.inf, -np.inf], np.nan, inplace=True)
            data_transformed[wrap_pow(col, power)] = transformed

    for f1, f2 in combinations(feature_list, 2):
        if is_near_duplicate(f1, f2):
            continue
        
        kind1 = get_interval_kind(f1)
        kind2 = get_interval_kind(f2)
        if {kind1, kind2} == {"average", "sum"}:
            continue
        
        c1 = extract_center(f1)
        c2 = extract_center(f2)
        tag1 = get_interval_tag(c1)
        tag2 = get_interval_tag(c2)
        if tag1 != tag2:
            continue

        numerator = data[f1].replace(0, zero_substitute)
        denominator = data[f2].replace(0, zero_substitute)
        ratio = numerator / denominator
        ratio.replace([np.inf, -np.inf], np.nan, inplace=True)

        data_transformed[wrap_div(f1, f2)] = ratio

    return data_transformed


### Features classification

In [12]:
def classify_feature_type(base_columns, columns):
    base_columns = [col for col in base_columns if "Unnamed" not in col]
    columns = [col for col in columns if "Unnamed" not in col]

    def clean(col):
        if col.startswith('$') and col.endswith('$'):
            col = col[1:-1]
        return col.replace("Σ", r"\Sigma")

    def is_interval(col):
        col = clean(col)
        return r"\overline{I}_{" in col or r"\Sigma I_{p," in col

    def is_peak(col):
        return not is_interval(col)

    def extract_numbers(col):
        return list(map(int, re.findall(r"\d{3,4}", col)))

    def is_in_range(col, low, high):
        return any(low <= num <= high for num in extract_numbers(clean(col)))

    def classify(col, base=True):
        col_clean = clean(col)
        source = "intervals" if is_interval(col_clean) else "peaks"

        if is_in_range(col_clean, 2500, 3800):
            region = "3800_2500"
        elif is_in_range(col_clean, 800, 1900):
            region = "1900_800"
        else:
            region = "unknown"

        kind = "base" if base else "engineered"
        return f"{kind}_features_{source}_{region}"

    groups = {}

    for col in base_columns:
        key = classify(col, base=True)
        groups.setdefault(key, []).append(col)

    for col in columns:
        if col not in base_columns:
            key = classify(col, base=False)
            groups.setdefault(key, []).append(col)

    groups["base_features"] = base_columns
    groups["engineered_features"] = [col for col in columns if col not in base_columns]

    return groups

## Calculation process

In [13]:
all_features = initial_features_df.columns.tolist()
features_1900_800_list = [c for c in all_features if get_interval_tag(extract_center(c)) == "low"]
features_3800_2500_list = [c for c in all_features if get_interval_tag(extract_center(c)) == "high"]

print(
    f"Число фичей:",
    f"(*) 1900–800: {len(features_1900_800_list)}",
    f"(*) 3800–2500: {len(features_3800_2500_list)}",
    sep='\n'
)

Число фичей:
(*) 1900–800: 22
(*) 3800–2500: 8


### 3800—2500 cm$^{-1}$

In [14]:
engineered_3800_2500 = generate_transformed_features(
    data=features_3800_2500, 
    feature_list=features_3800_2500_list, 
    tolerance=0
)

In [15]:
nan_df = engineered_3800_2500.isna().sum()
nan_summary = nan_df[nan_df > 0].sort_values(ascending=False)

print(f"Столбцов с NaN-values: {nan_summary.shape[0]}")
display(nan_summary.head(10))

Столбцов с NaN-values: 0


Series([], dtype: int64)

In [16]:
all_features_3800_2500 = pd.merge(
    left=features_3800_2500,
    right=engineered_3800_2500,
    how='inner',
    left_index=True,
    right_index=True
)

print(all_features_3800_2500.shape)
all_features_3800_2500.head(3)

(74, 45)


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3352}$,$(I_{3352})^{-1}$,$\log I_{3726}$,$(I_{3726})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3352 \pm 50}$,$(\overline{I}_{3352 \pm 50})^{-1}$,$\log \overline{I}_{3726 \pm 50}$,$(\overline{I}_{3726 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3352}}$,$\frac{I_{2928}}{I_{3726}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3352 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3352}}{I_{3726}}$,$\frac{I_{3352}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3352}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3352}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3352}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3726}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3726}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{3352 \pm 50}}{\overline{I}_{3726 \pm 50}}$
division_1_size_bulk,0.491720,1.0,0.088051,0.526201,1.579770,0.494443,0.983311,0.075225,-0.709847,2.033679,0.0,1.0,-2.429844,11.357112,-0.642072,1.900414,0.457279,0.633003,-0.704324,2.022478,-0.016830,1.016972,-2.587267,13.293390,0.491720,5.584514,0.934471,0.311260,0.500065,6.536620,11.357112,1.900414,0.633003,2.022478,13.293390,0.167333,0.055736,0.178080,0.089545,1.064230,0.535132,6.994997,0.502835,6.572822,13.071540
division_1_size_5,0.406937,1.0,0.028000,0.477909,1.434937,0.396989,0.980473,0.034739,-0.899098,2.457385,0.0,1.0,-3.575539,35.713875,-0.738335,2.092448,0.361121,0.696895,-0.923846,2.518960,-0.019720,1.019916,-3.359889,28.786000,0.406937,14.533284,0.851494,0.283592,0.415041,11.714078,35.713875,2.092448,0.696895,2.518960,28.786000,0.058589,0.019513,0.070532,0.028558,1.203834,0.487427,13.757094,0.404896,11.427734,28.223893
division_1_size_3,0.618445,1.0,0.078392,0.535789,1.696837,0.581733,0.980981,0.066886,-0.480547,1.616959,0.0,1.0,-2.546033,12.756398,-0.624015,1.866407,0.528766,0.589332,-0.541743,1.719000,-0.019202,1.019388,-2.704765,14.950801,0.618445,7.889130,1.154270,0.364469,0.630435,9.246247,12.756398,1.866407,0.589332,1.719000,14.950801,0.146311,0.046199,0.134756,0.079912,0.921021,0.546177,8.010474,0.593012,8.697381,14.666449


In [17]:
base_columns = features_3800_2500.columns.tolist()
columns = all_features_3800_2500.columns.tolist()
features_description_dict_3800_2500 = classify_feature_type(base_columns=base_columns, columns=columns)

for feature_type, feature_lst in features_description_dict_3800_2500.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_peaks_3800_2500: 3
(*) base_features_intervals_3800_2500: 5
(*) engineered_features_peaks_3800_2500: 9
(*) engineered_features_intervals_3800_2500: 28
(*) base_features: 8
(*) engineered_features: 37


### 1900—800 cm$^{-1}$

In [18]:
engineered_1900_800 = generate_transformed_features(
    data=features_1900_800, 
    feature_list=features_1900_800_list, 
    tolerance=0
)

In [19]:
nan_df = engineered_1900_800.isna().sum()
nan_summary = nan_df[nan_df > 0].sort_values(ascending=False)

print(f"Столбцов с NaN-values: {nan_summary.shape[0]}")
display(nan_summary.head(10))

Столбцов с NaN-values: 0


Series([], dtype: int64)

In [20]:
all_features_1900_800 = pd.merge(
    left=features_1900_800,
    right=engineered_1900_800,
    how='inner',
    left_index=True,
    right_index=True
)

print(all_features_1900_800.shape)
all_features_1900_800.head(3)

(74, 245)


,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$,$\log I_{872}$,$(I_{872})^{-1}$,$\log I_{1085}$,$(I_{1085})^{-1}$,$\log I_{1182}$,$(I_{1182})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1612}$,$(I_{1612})^{-1}$,$\log I_{1725}$,$(I_{1725})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{872 \pm 25}$,$(\overline{I}_{872 \pm 25})^{-1}$,$\log \overline{I}_{1085 \pm 25}$,$(\overline{I}_{1085 \pm 25})^{-1}$,$\log \overline{I}_{1182 \pm 25}$,$(\overline{I}_{1182 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1612 \pm 25}$,$(\overline{I}_{1612 \pm 25})^{-1}$,$\log \overline{I}_{1725 \pm 25}$,$(\overline{I}_{1725 \pm 25})^{-1}$,$\frac{I_{872}}{I_{1085}}$,$\frac{I_{872}}{I_{1182}}$,$\frac{I_{872}}{I_{1241}}$,$\frac{I_{872}}{I_{1394}}$,$\frac{I_{872}}{I_{1612}}$,$\frac{I_{872}}{I_{1725}}$,$\frac{I_{872}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{872}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{872}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1085 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1085}}{I_{1182}}$,$\frac{I_{1085}}{I_{1241}}$,$\frac{I_{1085}}{I_{1394}}$,$\frac{I_{1085}}{I_{1612}}$,$\frac{I_{1085}}{I_{1725}}$,$\frac{I_{1085}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1085}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1085}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{872 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1182}}{I_{1241}}$,$\frac{I_{1182}}{I_{1394}}$,$\frac{I_{1182}}{I_{1612}}$,$\frac{I_{1182}}{I_{1725}}$,$\frac{I_{1182}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1182}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1182}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1500\text

In [21]:
base_columns = features_1900_800.columns.tolist()
columns = all_features_1900_800.columns.tolist()
features_description_dict_1900_800 = classify_feature_type(base_columns=base_columns, columns=columns)

for feature_type, feature_lst in features_description_dict_1900_800.items():
    print(f'(*) {feature_type}: {len(feature_lst)}')

(*) base_features_peaks_1900_800: 7
(*) base_features_intervals_1900_800: 15
(*) engineered_features_peaks_1900_800: 35
(*) engineered_features_intervals_1900_800: 188
(*) base_features: 22
(*) engineered_features: 223


## Construction of datasets

### 3800—2500 cm$^{-1}$

In [22]:
dataset_3800_2500_part = dataset_3800_2500[['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class']]

print(dataset_3800_2500_part.shape)
dataset_3800_2500_part.head(3)

(74, 4)


,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,Train,$> 0$,$d > 5$,2
division_1_size_5,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,Train,$< 3$,$2 < d \leq 5$,1


In [23]:
dataset_3800_2500 = pd.merge(all_features_3800_2500, dataset_3800_2500_part, left_index=True, right_index=True, how='right')

print(dataset_3800_2500.shape)
display(dataset_3800_2500.head(3))
display(pd.DataFrame(dataset_3800_2500.isna().sum()).T)

(74, 49)


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3352}$,$(I_{3352})^{-1}$,$\log I_{3726}$,$(I_{3726})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3352 \pm 50}$,$(\overline{I}_{3352 \pm 50})^{-1}$,$\log \overline{I}_{3726 \pm 50}$,$(\overline{I}_{3726 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3352}}$,$\frac{I_{2928}}{I_{3726}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3352 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3352}}{I_{3726}}$,$\frac{I_{3352}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3352}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3352}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3352}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3726}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3726}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{3352 \pm 50}}{\overline{I}_{3726 \pm 50}}$,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,0.491720,1.0,0.088051,0.526201,1.579770,0.494443,0.983311,0.075225,-0.709847,2.033679,0.0,1.0,-2.429844,11.357112,-0.642072,1.900414,0.457279,0.633003,-0.704324,2.022478,-0.016830,1.016972,-2.587267,13.293390,0.491720,5.584514,0.934471,0.311260,0.500065,6.536620,11.357112,1.900414,0.633003,2.022478,13.293390,0.167333,0.055736,0.178080,0.089545,1.064230,0.535132,6.994997,0.502835,6.572822,13.071540,Train,$> 0$,$d > 5$,2
division_1_size_5,0.406937,1.0,0.028000,0.477909,1.434937,0.396989,0.980473,0.034739,-0.899098,2.457385,0.0,1.0,-3.575539,35.713875,-0.738335,2.092448,0.361121,0.696895,-0.923846,2.518960,-0.019720,1.019916,-3.359889,28.786000,0.406937,14.533284,0.851494,0.283592,0.415041,11.714078,35.713875,2.092448,0.696895,2.518960,28.786000,0.058589,0.019513,0.070532,0.028558,1.203834,0.487427,13.757094,0.404896,11.427734,28.223893,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,0.618445,1.0,0.078392,0.535789,1.696837,0.581733,0.980981,0.066886,-0.480547,1.616959,0.0,1.0,-2.546033,12.756398,-0.624015,1.866407,0.528766,0.589332,-0.541743,1.719000,-0.019202,1.019388,-2.704765,14.950801,0.618445,7.889130,1.154270,0.364469,0.630435,9.246247,12.756398,1.866407,0.589332,1.719000,14.950801,0.146311,0.046199,0.134756,0.079912,0.921021,0.546177,8.010474,0.593012,8.697381,14.666449,Train,$< 3$,$2 < d \leq 5$,1


,$I_{2928}$,$I_{3352}$,$I_{3726}$,$\overline{I}_{3800\text{–}2500}$,"$\Sigma I_{p,\ 3800\text{–}2500}$",$\overline{I}_{2928 \pm 50}$,$\overline{I}_{3352 \pm 50}$,$\overline{I}_{3726 \pm 50}$,$\log I_{2928}$,$(I_{2928})^{-1}$,$\log I_{3352}$,$(I_{3352})^{-1}$,$\log I_{3726}$,$(I_{3726})^{-1}$,$\log \overline{I}_{3800\text{–}2500}$,$(\overline{I}_{3800\text{–}2500})^{-1}$,"$\log \Sigma I_{p,\ 3800\text{–}2500}$","$(\Sigma I_{p,\ 3800\text{–}2500})^{-1}$",$\log \overline{I}_{2928 \pm 50}$,$(\overline{I}_{2928 \pm 50})^{-1}$,$\log \overline{I}_{3352 \pm 50}$,$(\overline{I}_{3352 \pm 50})^{-1}$,$\log \overline{I}_{3726 \pm 50}$,$(\overline{I}_{3726 \pm 50})^{-1}$,$\frac{I_{2928}}{I_{3352}}$,$\frac{I_{2928}}{I_{3726}}$,$\frac{I_{2928}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{2928}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{2928}}{\overline{I}_{3352 \pm 50}}$,$\frac{I_{2928}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3352}}{I_{3726}}$,$\frac{I_{3352}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3352}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3352}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3352}}{\overline{I}_{3726 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3800\text{–}2500}}$,"$\frac{I_{3726}}{\Sigma I_{p,\ 3800\text{–}2500}}$",$\frac{I_{3726}}{\overline{I}_{2928 \pm 50}}$,$\frac{I_{3726}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{2928 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{3800\text{–}2500}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3352 \pm 50}}$,$\frac{\overline{I}_{2928 \pm 50}}{\overline{I}_{3726 \pm 50}}$,$\frac{\overline{I}_{3352 \pm 50}}{\overline{I}_{3726 \pm 50}}$,Split,Fraction_hue,Fraction_grouped_hue,Class
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### 1900—800 cm$^{-1}$

In [24]:
dataset_1900_800_part = dataset_1900_800[['Split', 'Fraction_hue', 'Fraction_grouped_hue', 'Class']]

print(dataset_1900_800_part.shape)
dataset_1900_800_part.head(3)

(74, 4)


,Split,Fraction_hue,Fraction_grouped_hue,Class
division_1_size_bulk,Train,$> 0$,$d > 5$,2
division_1_size_5,Train,$< 5$,$2 < d \leq 5$,1
division_1_size_3,Train,$< 3$,$2 < d \leq 5$,1


In [25]:
dataset_1900_800= pd.merge(all_features_1900_800, dataset_1900_800_part, left_index=True, right_index=True, how='right')

print(dataset_1900_800.shape)
display(dataset_1900_800.head(3))
display(pd.DataFrame(dataset_1900_800.isna().sum()).T)

(74, 249)


,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$,$\log I_{872}$,$(I_{872})^{-1}$,$\log I_{1085}$,$(I_{1085})^{-1}$,$\log I_{1182}$,$(I_{1182})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1612}$,$(I_{1612})^{-1}$,$\log I_{1725}$,$(I_{1725})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{872 \pm 25}$,$(\overline{I}_{872 \pm 25})^{-1}$,$\log \overline{I}_{1085 \pm 25}$,$(\overline{I}_{1085 \pm 25})^{-1}$,$\log \overline{I}_{1182 \pm 25}$,$(\overline{I}_{1182 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1612 \pm 25}$,$(\overline{I}_{1612 \pm 25})^{-1}$,$\log \overline{I}_{1725 \pm 25}$,$(\overline{I}_{1725 \pm 25})^{-1}$,$\frac{I_{872}}{I_{1085}}$,$\frac{I_{872}}{I_{1182}}$,$\frac{I_{872}}{I_{1241}}$,$\frac{I_{872}}{I_{1394}}$,$\frac{I_{872}}{I_{1612}}$,$\frac{I_{872}}{I_{1725}}$,$\frac{I_{872}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{872}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{872}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1085 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1085}}{I_{1182}}$,$\frac{I_{1085}}{I_{1241}}$,$\frac{I_{1085}}{I_{1394}}$,$\frac{I_{1085}}{I_{1612}}$,$\frac{I_{1085}}{I_{1725}}$,$\frac{I_{1085}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1085}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1085}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{872 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1182}}{I_{1241}}$,$\frac{I_{1182}}{I_{1394}}$,$\frac{I_{1182}}{I_{1612}}$,$\frac{I_{1182}}{I_{1725}}$,$\frac{I_{1182}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1182}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1182}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1500\text

,$I_{872}$,$I_{1085}$,$I_{1182}$,$I_{1241}$,$I_{1394}$,$I_{1612}$,$I_{1725}$,$\overline{I}_{1900\text{–}800}$,"$\Sigma I_{p,\ 1900\text{–}800}$",$\overline{I}_{1900\text{–}1500}$,"$\Sigma I_{p,\ 1900\text{–}1500}$",$\overline{I}_{1500\text{–}1300}$,"$\Sigma I_{p,\ 1500\text{–}1300}$",$\overline{I}_{1300\text{–}800}$,"$\Sigma I_{p,\ 1300\text{–}800}$",$\overline{I}_{872 \pm 25}$,$\overline{I}_{1085 \pm 25}$,$\overline{I}_{1182 \pm 25}$,$\overline{I}_{1241 \pm 25}$,$\overline{I}_{1394 \pm 25}$,$\overline{I}_{1612 \pm 25}$,$\overline{I}_{1725 \pm 25}$,$\log I_{872}$,$(I_{872})^{-1}$,$\log I_{1085}$,$(I_{1085})^{-1}$,$\log I_{1182}$,$(I_{1182})^{-1}$,$\log I_{1241}$,$(I_{1241})^{-1}$,$\log I_{1394}$,$(I_{1394})^{-1}$,$\log I_{1612}$,$(I_{1612})^{-1}$,$\log I_{1725}$,$(I_{1725})^{-1}$,$\log \overline{I}_{1900\text{–}800}$,$(\overline{I}_{1900\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}800}$","$(\Sigma I_{p,\ 1900\text{–}800})^{-1}$",$\log \overline{I}_{1900\text{–}1500}$,$(\overline{I}_{1900\text{–}1500})^{-1}$,"$\log \Sigma I_{p,\ 1900\text{–}1500}$","$(\Sigma I_{p,\ 1900\text{–}1500})^{-1}$",$\log \overline{I}_{1500\text{–}1300}$,$(\overline{I}_{1500\text{–}1300})^{-1}$,"$\log \Sigma I_{p,\ 1500\text{–}1300}$","$(\Sigma I_{p,\ 1500\text{–}1300})^{-1}$",$\log \overline{I}_{1300\text{–}800}$,$(\overline{I}_{1300\text{–}800})^{-1}$,"$\log \Sigma I_{p,\ 1300\text{–}800}$","$(\Sigma I_{p,\ 1300\text{–}800})^{-1}$",$\log \overline{I}_{872 \pm 25}$,$(\overline{I}_{872 \pm 25})^{-1}$,$\log \overline{I}_{1085 \pm 25}$,$(\overline{I}_{1085 \pm 25})^{-1}$,$\log \overline{I}_{1182 \pm 25}$,$(\overline{I}_{1182 \pm 25})^{-1}$,$\log \overline{I}_{1241 \pm 25}$,$(\overline{I}_{1241 \pm 25})^{-1}$,$\log \overline{I}_{1394 \pm 25}$,$(\overline{I}_{1394 \pm 25})^{-1}$,$\log \overline{I}_{1612 \pm 25}$,$(\overline{I}_{1612 \pm 25})^{-1}$,$\log \overline{I}_{1725 \pm 25}$,$(\overline{I}_{1725 \pm 25})^{-1}$,$\frac{I_{872}}{I_{1085}}$,$\frac{I_{872}}{I_{1182}}$,$\frac{I_{872}}{I_{1241}}$,$\frac{I_{872}}{I_{1394}}$,$\frac{I_{872}}{I_{1612}}$,$\frac{I_{872}}{I_{1725}}$,$\frac{I_{872}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{872}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{872}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{872}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{872}}{\overline{I}_{1085 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{872}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1085}}{I_{1182}}$,$\frac{I_{1085}}{I_{1241}}$,$\frac{I_{1085}}{I_{1394}}$,$\frac{I_{1085}}{I_{1612}}$,$\frac{I_{1085}}{I_{1725}}$,$\frac{I_{1085}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1085}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1500\text{–}1300}}$",$\frac{I_{1085}}{\overline{I}_{1300\text{–}800}}$,"$\frac{I_{1085}}{\Sigma I_{p,\ 1300\text{–}800}}$",$\frac{I_{1085}}{\overline{I}_{872 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1182 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1241 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1394 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1612 \pm 25}}$,$\frac{I_{1085}}{\overline{I}_{1725 \pm 25}}$,$\frac{I_{1182}}{I_{1241}}$,$\frac{I_{1182}}{I_{1394}}$,$\frac{I_{1182}}{I_{1612}}$,$\frac{I_{1182}}{I_{1725}}$,$\frac{I_{1182}}{\overline{I}_{1900\text{–}800}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}800}}$",$\frac{I_{1182}}{\overline{I}_{1900\text{–}1500}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1900\text{–}1500}}$",$\frac{I_{1182}}{\overline{I}_{1500\text{–}1300}}$,"$\frac{I_{1182}}{\Sigma I_{p,\ 1500\text

# Saving data

In [26]:
dataset_3800_2500.to_csv('./../data/processed_data/dataset_3800_2500_engineered.csv', sep=';')
dataset_1900_800.to_csv('./../data/processed_data/dataset_1900_800_engineered.csv', sep=';')

In [27]:
with open('./../data/processed_data/features_description_3800_2500.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict_3800_2500, f, ensure_ascii=False, indent=2)

In [28]:
with open('./../data/processed_data/features_description_1900_800.json', 'w', encoding='utf-8') as f:
    json.dump(features_description_dict_1900_800, f, ensure_ascii=False, indent=2)